In [ ]:
# ---------------------------------------------THE BEST EDITION OF THE FLIPPER CONVERTER----------------------------------------------
from PIL import Image
import os
import re
import struct
from typing import List, Tuple, NamedTuple, Union

# --- CONFIGURATION CONSTANTS ---
FLIPPER_WIDTH = 128
FLIPPER_HEIGHT = 64
NUM_PAGES = FLIPPER_HEIGHT // 8 # 8 pages
EXPECTED_DATA_LENGTH = (FLIPPER_WIDTH * FLIPPER_HEIGHT) // 8 # 1024 bytes (for 128x64)
BM_HEADER_BYTE = 0x0
FIXED_THRESHOLD = 128
# -----------------------------

# --- CONFIGURATION STRUCTURE ---
class FlipperConfig(NamedTuple):
    """Encapsulates all conversion parameters for clarity."""
    col_major: bool             # Must be True for Flipper
    msb_first: bool             # Bit order within the byte (page)
    reverse_x: bool             # Should column order be reversed (Achieves Y-axis reflection via data order)
    invert_colors: bool         # False: Black in PNG -> LIT (Bit=1, Orange). True: Black in PNG -> UNLIT (Bit=0, Transparent)
    flip_vertical: bool         # Apply vertical flip (X-axis reflection)
    flip_horizontal: bool       # Apply horizontal flip (Y-axis reflection)
    crop_left_half: bool        # Crop 64x64 left half and stretch to 128x64
    rotate_clockwise: bool      # Apply 90-degree rotation (Behavior updated to CCW below)

# --- CORE PARAMETER SETTING (Y-AXIS REFLECTION IMPLEMENTED VIA reverse_x=True) ---
DEFAULT_CONFIG = FlipperConfig(
    col_major=True,
    msb_first=False,
    reverse_x=True,             # <--- SET TO TRUE for Y-axis reflection via reversed data order
    invert_colors=False,
    flip_vertical=False,
    flip_horizontal=False,      # Disabled as reverse_x handles the flip during data generation
    crop_left_half=False,
    rotate_clockwise=True       # ENABLED for 90-degree counter-clockwise rotation
)
# ------------------------------


def natural_sort_key(s: str) -> List[Union[int, str]]:
    """Key for natural sorting (e.g., 'frame_10.png' comes after 'frame_9.png')."""
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split('([0-9]+)', s)]

def process_png_file(
    png_path: str,
    do_flip_vertical: bool,
    do_flip_horizontal: bool, # Parameter for Y-axis reflection
    do_crop_left: bool,
    do_rotate_clockwise: bool, # Rotation parameter
    target_size: Tuple[int, int]=(FLIPPER_WIDTH, FLIPPER_HEIGHT)
) -> Union[Image.Image, None]:
    """
    Opens a PNG, converts it to 1-bit, applies cropping/flipping/rotation based on config.
    """
    try:
        img = Image.open(png_path)
    except FileNotFoundError:
        print(f"Error: PNG file not found at {png_path}")
        return None
    except Exception as e:
        print(f"Error opening image {png_path}: {e}")
        return None

    # Ensure the image is 128x64 or resize it.
    if img.size != target_size:
        print(f"Warning: Image {png_path} is {img.size}. Resizing to required 128x64.")
        img = img.resize(target_size, resample=Image.Resampling.LANCZOS)
    else:
        print(f"  Image {png_path} confirmed 128x64.")

    frame_full = img # The image is now guaranteed 128x64.

    # --- Rotation Logic ---
    if do_rotate_clockwise:
        # 1. Rotate 90 degrees counter-clockwise (128x64 becomes 64x128)
        print("  Applying 90-degree counter-clockwise rotation.")
        frame_full = frame_full.transpose(Image.ROTATE_270) # <--- Rotates 90 degrees CCW
        
        # 2. Resize back to 128x64 to fit the Flipper screen size and maintain format consistency
        print("  Resizing rotated 64x128 frame back to 128x64 (WARNING: This will cause stretch/distortion).")
        frame_full = frame_full.resize(target_size, resample=Image.Resampling.LANCZOS)
    # ----------------------

    if do_crop_left:
        # This block handles the 64x64 cropping/stretching logic if enabled.
        crop_box = (0, 0, 64, 64)
        print(f"  Applying crop to left half (0, 0, 64, 64).")
        frame_cropped = frame_full.crop(crop_box)

        # Resize the cropped 64x64 image back to 128x64 to fill the screen.
        print(f"  Resizing 64x64 crop back to 128x64 to fill the screen.")
        frame_full = frame_cropped.resize(target_size, resample=Image.Resampling.LANCZOS)

    if do_flip_vertical:
        print("  Applying vertical flip (X-axis reflection) to the frame.")
        frame_full = frame_full.transpose(Image.FLIP_TOP_BOTTOM)

    if do_flip_horizontal: # Y-AXIS REFLECTION IMPLEMENTATION
        # This is now primarily controlled by 'reverse_x' in the data generation
        print("  Applying horizontal flip (Y-axis reflection) to the frame via PIL.")
        frame_full = frame_full.transpose(Image.FLIP_LEFT_RIGHT)

    # 1. Convert to Grayscale (L)
    frame_grayscale = frame_full.convert('L')

    # 2. Apply fixed threshold (NON-DITHERED) and convert to '1' mode (Black=0, White=255)
    frame_monochrome = frame_grayscale.point(
        lambda x: 0 if x < FIXED_THRESHOLD else 255,
        mode='1'
    )

    return frame_monochrome


def generate_flipper_data(image_frame: Image.Image, invert_colors: bool, msb_first: bool, reverse_x_order: bool) -> bytearray:
    """
    Generates the full Flipper .bm byte data (1024 bytes) in Column-Major format.
    The reverse_x_order parameter implements the Y-axis reflection by iterating columns backward.
    """

    byte_data = bytearray()

    # Define the range of columns (X-axis)
    if reverse_x_order:
        # X runs from 127 down to 0 (Achieves Y-axis reflection)
        x_range = range(FLIPPER_WIDTH - 1, -1, -1)
        print("  --- INFO: Columns are being reversed (127 -> 0) during data generation. ---")
    else:
        # X runs from 0 up to 127
        x_range = range(0, FLIPPER_WIDTH)

    # --- Column-Major (standard Flipper format) ---
    # Outer loop iterates through columns (X-axis)
    for x in x_range:
        # Inner loop iterates through pages (Y-groups, 8 pages total)
        for page in range(0, NUM_PAGES):
            byte = 0

            # Process 8 pixels (Y-axis for the current byte/page)
            for y_offset in range(0, 8):
                # Calculate the global Y coordinate
                y = (page * 8) + y_offset

                # Get the pixel value (0=Black, 255=White in PIL '1' mode)
                pixel_value = image_frame.getpixel((x, y))

                # --- COLOR LOGIC (Bit '1' means LIT/Orange) ---
                is_black = (pixel_value == 0)

                if invert_colors:
                    # Invert: Black Logo (0) -> UNLIT Bit (0). White BG (255) -> LIT Bit (1)
                    should_set_bit = not is_black
                else:
                    # Non-Invert (Flipper Default):
                    # Black Logo (0) -> LIT Bit (1). White BG (255) -> UNLIT Bit (0)
                    should_set_bit = is_black

                if should_set_bit:
                    # Set the bit based on MSB or LSB order
                    # MSB First: bit_pos = (7 - y_offset) (Bit 7 is top pixel, y=0)
                    # LSB First: bit_pos = y_offset      (Bit 0 is top pixel, y=0)
                    bit_pos = (7 - y_offset) if msb_first else y_offset
                    byte |= (1 << bit_pos)

            byte_data.append(byte)

    return byte_data # Must be exactly 1024 bytes


def convert_to_flipper_bm(image_frame: Image.Image, invert_colors: bool, msb_first: bool, reverse_x_order: bool) -> bytearray:
    """
    Generates the 1025-byte Flipper Zero .bm file (1 byte header + 1024 bytes data).
    """

    if image_frame.size != (FLIPPER_WIDTH, FLIPPER_HEIGHT):
        print("Error: Image must be 128x64 pixels after processing.")
        return bytearray()

    final_data = generate_flipper_data(image_frame, invert_colors, msb_first, reverse_x_order)

    if len(final_data) != EXPECTED_DATA_LENGTH: # Check for 1024 bytes
        print(f"CRITICAL ERROR: Generated data is {len(final_data)} bytes, expected {EXPECTED_DATA_LENGTH}.")
        return bytearray()

    # Add the 1-byte header
    return bytearray([BM_HEADER_BYTE]) + final_data


def save_bm_as_png(
    bm_data: bytearray,
    width: int,
    height: int,
    output_filename: str,
    invert_colors: bool,
    msb_first: bool
) -> Image.Image:
    """
    Converts the Flipper Zero .bm byte data back into a viewable PNG image for verification.
    """
    raw_data = bm_data[1:] # Remove the 1-byte header
    png_image = Image.new('1', (width, height), color=255) # Start with White background
    data_expected_length = (width * height) // 8 # 1024 bytes

    if len(raw_data) != data_expected_length:
        print(f"Warning: Preview received {len(raw_data)} bytes, expected {data_expected_length}. Proceeding but output may be incorrect.")
        return png_image

    display_data = raw_data

    # --- 1. Process bytes in L-R display order (Column-Major index order) ---
    for byte_index, byte in enumerate(display_data):

        # Calculate the X, Y coordinates corresponding to this byte
        # x is the column index (0 to 127)
        x = byte_index // NUM_PAGES
        # data_page is the page index (0 to 7)
        data_page = byte_index % NUM_PAGES

        # --- 2. Process 8 pixels (Y-axis for the current byte/page) ---
        for y_offset in range(0, 8):
            y = (data_page * 8) + y_offset

            # Determine which bit position corresponds to 'y_offset' based on LSB/MSB
            bit_position = (7 - y_offset) if msb_first else y_offset

            # Check the bit (1 if LIT/Orange)
            is_set = (byte >> bit_position) & 1

            # --- Reverse Color Logic for PNG preview ---
            should_be_black_in_png = False

            if invert_colors:
                # Invert Conversion: Bit '1' in BM means LIT (White/BG). Bit '0' in BM means UNLIT (Black/FG)
                if is_set == 0:
                    should_be_black_in_png = True
            else:
                # Non-Invert Conversion (Flipper Default):
                # Bit '1' in BM means LIT (Black/FG). Bit '0' in BM means UNLIT (White/BG)
                if is_set == 1:
                    should_be_black_in_png = True

            if should_be_black_in_png:
                # Set the pixel to BLACK (0) in the PNG preview
                png_image.putpixel((x, y), 0)

    png_image.save(output_filename)
    return png_image


def convert_png_folder_to_flipper_bm_frames(input_folder: str, output_prefix: str, config: FlipperConfig) -> int:
    """
    Main function to convert all PNGs in a folder to Flipper .bm files and save a cohesive GIF preview.
    """

    EXPECTED_FILE_SIZE = EXPECTED_DATA_LENGTH + 1 # 1025 bytes

    print(f"\n--- Starting conversion of PNGs in {input_folder} ---")

    color_mode = "NON-INVERTED (Bit 1 = Black/Lit)" if not config.invert_colors else "INVERTED (Bit 1 = White/Lit)"
    bit_order = "MSB First" if config.msb_first else "LSB First"
    x_order = "REVERSED (127 down to 0) - Y-Axis Reflected" if config.reverse_x else "NORMAL (0 up to 127)"
    major_mode = "Column-Major (Enforced)"
    flip_mode_v = "X-AXIS FLIP ON" if config.flip_vertical else "NO X-AXIS FLIP"
    flip_mode_h = "Y-AXIS FLIP ON (via PIL)" if config.flip_horizontal else "NO Y-AXIS FLIP (Handled by reverse_x)"
    crop_mode = "CROP LEFT HALF ON" if config.crop_left_half else "NO CROPPING (Full Resizing)"
    rotate_mode = "90 DEGREE COUNTER-CLOCKWISE ROTATE ON" if config.rotate_clockwise else "NO ROTATION" # <--- Updated description
    data_order = f"128x64 FULL FILE OUTPUT"

    print(f"  Mode: {major_mode}, Color: {color_mode}, Bit Order: {bit_order}, X-Order: {x_order}, Flip V: {flip_mode_v}, Flip H: {flip_mode_h}, Crop: {crop_mode}, Rotate: {rotate_mode}")
    print(f"  Data Output Order: {data_order}")

    png_files = [f for f in os.listdir(input_folder) if f.lower().endswith('.png')]
    png_files.sort(key=natural_sort_key)

    if not png_files:
        print("Error: No PNG files found in the source folder.")
        return 0

    frame_index = 0
    png_files_to_delete = []
    preview_frames = []

    for filename in png_files:
        full_path = os.path.join(input_folder, filename)

        # Pass necessary config fields to process_png_file
        frame_image = process_png_file(
            full_path,
            config.flip_vertical,
            config.flip_horizontal, # Pass horizontal flip parameter
            config.crop_left_half,
            config.rotate_clockwise # Pass rotation parameter
        )
        if frame_image is None:
            continue

        print(f"Processing frame: {filename}")
        bm_data = convert_to_flipper_bm(
            frame_image,
            invert_colors=config.invert_colors,
            msb_first=config.msb_first,
            reverse_x_order=config.reverse_x # Pass the reverse_x_order flag for data generation
        )

        if not bm_data:
            print(f"Skipping frame {filename} due to data generation error.")
            continue

        if len(bm_data) != EXPECTED_FILE_SIZE:
            print(f"CRITICAL ERROR: Final BM data size is {len(bm_data)} bytes. Expected {EXPECTED_FILE_SIZE} bytes.")
            continue
        else:
            print(f"SUCCESS: Data size for {filename} is correctly {len(bm_data)} bytes.")

        output_bm_filename = os.path.join(input_folder, f"{output_prefix}_{frame_index:03d}.bm")
        output_png_filename = os.path.join(input_folder, f"preview_{output_prefix}_{frame_index:03d}.png")

        # Save .bm file
        with open(output_bm_filename, 'wb') as f:
            f.write(bm_data)

        # Save viewable PNG preview (returns the PIL Image object)
        png_image = save_bm_as_png(
            bm_data,
            FLIPPER_WIDTH,
            FLIPPER_HEIGHT,
            output_png_filename,
            invert_colors=config.invert_colors,
            msb_first=config.msb_first
        )
        preview_frames.append(png_image)
        png_files_to_delete.append(output_png_filename)

        frame_index += 1

    # --- Stitch all PNG frames into a single GIF ---
    if preview_frames:
        preview_gif_name = os.path.join(input_folder, f'{output_prefix}_preview.gif')
        # Use duration=100ms (10 frames per second)
        preview_frames[0].save(
            preview_gif_name,
            save_all=True,
            append_images=preview_frames[1:],
            duration=100,
            loop=0
        )
        print(f"  Generated '{preview_gif_name}' for visual verification.")

    # --- Cleanup ---
    for f in png_files_to_delete:
        os.remove(f)

    print(f"Conversion complete. Total frames saved: {frame_index}. Look for {output_prefix}_000.bm etc. in {input_folder}")
    return frame_index

# --- MAIN EXECUTION ---
if __name__ == '__main__':

    # !!! IMPORTANT: SET YOUR FOLDER PATH HERE !!!
    SOURCE_FOLDER = r"C:\Users\ericd\dolphin\flipper_animations\CNBC_128x64\cnbc"
    # The output prefix confirms the Y-axis reflection and 90-degree counter-clockwise rotation are applied
    OUTPUT_BM_PREFIX = "frame" 

    if os.path.isdir(SOURCE_FOLDER):
        convert_png_folder_to_flipper_bm_frames(
            SOURCE_FOLDER,
            OUTPUT_BM_PREFIX,
            config=DEFAULT_CONFIG # Pass the single configuration object
        )
    else:
        print(f"Error: Source folder not found at '{SOURCE_FOLDER}'. Please create the folder and place your 128x64 PNG files inside, then update the SOURCE_FOLDER variable in the script.")
